# Type Erasure

The expression could refer to any technique aimed at hiding a type. In the context of modern C++, it refers to a specific technique to replace a type parameter by object polymorphism, without the end user needing to be aware of it. We describe it below.

## Motivation

Let's imagine one wants to write some generic code, able to deal with any kind of particle, such as:
```
void print( Particle const & p )
 { std::cout<<p.name()<<std::endl ; }
```

The good old object-oriented solution would be to define some common abstract with virtual member functions:

In [16]:
%%file tmp.type-erasure.cpp

#include <iostream>
#include <string_view>
#include <vector>
#include <ctime>

struct Particle
{
 virtual ~Particle() {}
 virtual std::string_view name() const = 0 ;
} ;

struct Electron : public Particle { std::string_view name() const { return "Electron" ; } } ;
struct Proton : public Particle { std::string_view name() const { return "Proton" ; } } ;
struct Neutron : public Particle{ std::string_view name() const { return "Neutron" ; } } ;

void print( Particle const & p )
 { std::cout<<p.name()<<std::endl ; }
 
Particle * new_particle()
 {
  const int NB = 3 ;
  int num = NB*(std::rand()/(double(RAND_MAX)+1)) ;
  switch (num)
   {
    case 0: return new Electron() ;
    case 1: return new Proton() ;
    case 2: return new Neutron() ;
   }
  return 0 ;
 }
 
int main()
{
 std::srand(time(0)) ;
 std::vector<Particle *> particles ;
 particles.push_back(new_particle()) ;
 particles.push_back(new_particle()) ;
 particles.push_back(new_particle()) ;

 for ( auto * p : particles )
  { print(*p) ; }
}

Overwriting tmp.type-erasure.cpp


In [17]:
!rm -f tmp.type-erasure.exe && g++ -std=c++17 tmp.type-erasure.cpp -o tmp.type-erasure.exe

In [23]:
!./tmp.type-erasure.exe

Electron
Neutron
Electron


Drawbacks of this approach:
- sometimes you would like to include types which are not under your control, and which you cannot make inherit from your abstract class `Particle`.
- the call to `name()` is slow, because it is indirectly solved at runtime.

Well, if all your classes comply to some implicit interface (the `name()` function), the template feature can do the job : 

In [1]:
%%file tmp.type-erasure.cpp

#include <iostream>
#include <string_view>

struct Electron { std::string_view name() const { return "Electron" ; } } ;
struct Proton { std::string_view name() const { return "Proton" ; } } ;
struct Neutron { std::string_view name() const { return "Neutron" ; } } ;

template< typename Particle >
void print( Particle const & p )
 { std::cout<<p.name()<<std::endl ; }
 
int main()
{
 print(Electron{}) ;
 print(Proton{}) ;
 print(Neutron{}) ;
}

The function `print()` can now deal with any class which has a `name()` member function... but there are obvious drawbacks:
- no simple way to put all particles in a `std::vector`
- no way to choose the type of each particle at runtime

And a less obvious drawback : once you start to add template parameters to a function or class, any client code will need to add this same parameters. The templates parameters naturally propagate to all the client code, hence leading to usual templates sores (code bloat, compilation time, error messages).

None of the two approachs above are perfect. If you need the flexibility of object-oriented polymorphism, yet you want to be able to deal with any type which comply to a given implicit interface, the type erasure idiom is the solution. Furthermore, that's a way to write some code which is generic, but do not exhibit any type paramter. Hence the name: *type erasure*. 

## Wrap external classes into an object-oriented inheritance tree

Let's say we start from the template code above, with all the particles classes as independant types. It is rather easy (and cumbersome) to create wrapper classes so that one can benefit from the object-oriented polymorphism.

In [34]:
%%file tmp.type-erasure.cpp

#include <iostream>
#include <string_view>
#include <vector>
#include <ctime>

struct Electron { std::string_view name() const { return "Electron" ; } } ;
struct Proton { std::string_view name() const { return "Proton" ; } } ;
struct Neutron { std::string_view name() const { return "Neutron" ; } } ;

struct Particle
{
 virtual ~Particle() {}
 virtual std::string_view name() const = 0 ;
} ;

template< typename T >
struct ParticleImpl : public Particle
{
 T m_obj ;
 ParticleImpl( T const & a_obj ) : m_obj(a_obj) {}
 std::string_view name() const { return m_obj.name() ; }
} ;

void print( Particle const & p )
 { std::cout<<p.name()<<std::endl ; }
 
Particle * new_particle()
 {
  const int NB = 3 ;
  int num = NB*(std::rand()/(double(RAND_MAX)+1)) ;
  switch (num)
   {
    case 0: return new ParticleImpl<Electron>{Electron{}} ;
    case 1: return new ParticleImpl<Proton>{Proton{}} ;
    case 2: return new ParticleImpl<Neutron>{Neutron{}} ;
   }
  return 0 ;
 }
 
int main()
{
 std::srand(time(0)) ;
 std::vector<Particle *> particles ;
 particles.push_back(new_particle()) ;
 particles.push_back(new_particle()) ;
 particles.push_back(new_particle()) ;

 for ( auto * p : particles )
  { print(*p) ; }
}

Overwriting tmp.type-erasure.cpp


In [35]:
!rm -f tmp.type-erasure.exe && g++ -std=c++17 tmp.type-erasure.cpp -o tmp.type-erasure.exe

In [36]:
!./tmp.type-erasure.exe

Neutron
Electron
Electron


## Hide everything

The original multiples types (`Electron`, `Proton` and `Neutron`) can now be used by a client code which only depends on a single type (`Particle`). The original types are somehow "erased". Yet, the object-oriented polymorphism is still very visible : `print()` must take as argument a pointer or a reference, the `vector` is a vector of pointers, etc. A typical "type erasure" implementation usually hides everything within a class which appears to have a simple value semantic. 

In [ ]:
#include <iostream>
#include <string_view>
#include <vector>
#include <ctime>

struct Electron { std::string_view name() const { return "Electron" ; } } ;
struct Proton { std::string_view name() const { return "Proton" ; } } ;
struct Neutron { std::string_view name() const { return "Neutron" ; } } ;

class Particle
{
  public :

    template< typename T >
    Particle( T const & a_obj ) : m_pc(new ParticleModel<T>(a_obj)) {}
    Particle( Particle const & a_p ) : m_pc(a_p.m_pc->clone()) {}
    void operator=( Particle const & a_p ) { delete m_pc ; m_pc = a_p.m_pc->clone() ; }
    Particle( Particle && a_p ) : m_pc(a_p.m_pc) { a_p.m_pc = nullptr ; }
    void operator=( Particle && a_p ) { delete m_pc ; m_pc = a_p.m_pc ; a_p.m_pc = nullptr ; }
    ~Particle() { delete m_pc ; }
    
    std::string_view name() const { return m_pc->name() ; }
  
  private :
  
    struct ParticleConcept
    {
     virtual ~ParticleConcept() {}
     virtual std::string_view name() const = 0 ;
     virtual ParticleConcept * clone() const = 0 ;
    } ;
  
    template< typename T >
    struct ParticleModel : public ParticleConcept
    {
     T m_obj ;
     ParticleModel( T const & a_obj ) : m_obj(a_obj) {}
     std::string_view name() const { return m_obj.name() ; }
     ParticleConcept * clone() const { return new ParticleModel<T>(m_obj) ; }
    } ;
    
    ParticleConcept * m_pc = nullptr ;
  
} ;

void print( Particle const & p )
 { std::cout<<p.name()<<std::endl ; }
 
Particle random_particle()
 {
  const int NB = 3 ;
  int num = NB*(std::rand()/(double(RAND_MAX)+1)) ;
  switch (num)
   {
    case 0: return Particle{Electron{}} ;
    case 1: return Particle{Proton{}} ;
    case 2: return Particle{Neutron{}} ;
   }
  throw "unexpected" ;
 }
 
int main()
{
 std::srand(time(0)) ;
 
 std::vector<Particle> particles ;
 particles.push_back(random_particle()) ;
 particles.push_back(random_particle()) ;
 particles.push_back(random_particle()) ;

 for ( auto const & p : particles )
  { print(p) ; }
}

In [ ]:
!rm -f tmp.type-erasure.exe && g++ -std=c++17 tmp.type-erasure.cpp -o tmp.type-erasure.exe

In [ ]:
!./tmp.type-erasure.exe

## To remember

## Exercise

## References

- [Dave Kilian](http://davekilian.com/cpp-type-erasure.html)
- [Sean Parent](https://youtu.be/bIhUE5uUFOA)
- [Andrzej Krzemieński](https://akrzemi1.wordpress.com/2013/11/18/type-erasure-part-i/)

© *CNRS 2022*  
*Assembled and written in french by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)